In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from copy import deepcopy
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sys 
import time

BATCH_SIZE = 10

print(tf.__version__)

2.7.0


In [ ]:
path_training ="/content/drive/MyDrive/IC-Colab-CSV files/01_Atributos_Weve.csv"
path_test ="/content/drive/MyDrive/IC-Colab-CSV files/02_Atributos_Weve.csv"
df = pd.read_csv(path_training)
df_test = pd.read_csv(path_test)
df


,ServerPort,ClientPort,10,15,17,22,24,29,31,32,33,34,36,37,38,45,46,47,48,59,60,61,62,63,64,75,76,77,78,79,80,81,82,83,84,87,88,89,90,91,92,alvos
0,80,16945,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,140,1,1,1,1,1,2,0,0,0,0,1380,1380,319,140,319,140,6900,61440,512,8192,0,0,WWW
1,80,16913,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,141,1,1,1,1,1,2,0,0,0,0,1380,1380,319,141,319,141,6900,61440,512,8192,0,0,WWW
2,80,16917,60,376,40,362,14,20,5,5,4,5,1,0,0,1,1,322,141,1,1,1,1,1,2,0,0,0,0,1380,1380,322,141,322,141,6900,61440,512,8192,0,0,WWW
3,80,17018,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,142,1,1,1,1,1,2,0,0,0,0,1380,1380,319,142,319,142,6900,61440,512,8192,0,0,WWW
4,80,16926,60,379,40,365,14,20,5,5,4,5,1,0,0,1,1,325,141,1,1,1,1,1,2,0,0,0,0,1380,1380,325,141,325,141,6900,61440,512,8192,0,0,WWW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24858,25,3707,60,139,40,125,14,20,9,10,8,10,4,0,0,3,4,83,163,3,4,1,1,1,1,0,0,0,0,1460,1460,35,85,20,8,64240,49640,64077,49640,0,0,MAIL
24859,25,58489,66,1061,52,1047,14,14,11,12,10,12,3,0,0,7,7,1197,318,7,7,1,1,1,1,0,0,0,0,1460,1460,995,73,6,21,5840,57920,5840,57920,0,0,MAIL
24860,25,4582,66,163,52,149,14,14,10,13,9,13,4,0,0,5,7,97,258,5,7,1,1,1,1,0,0,0,0,1460,1460,35,97,6,8,57920,49232,57911,49232,0,0,MAIL
24861,80,56031,60,590,40,576,14,20,28,52,27,52,6,0,0,1,45,119,23914,1,1,1,1,1,1,0,0,0,0,1460,1380,119,536,119,330,16080,61440,5840,8192,0,0,WWW


In [ ]:
#df.dtypes

In [ ]:
target = df.pop('alvos')
server_port = df.pop('ServerPort')

target_test = df_test.pop('alvos')
server_port_test = df_test.pop('ServerPort')

df

,ClientPort,10,15,17,22,24,29,31,32,33,34,36,37,38,45,46,47,48,59,60,61,62,63,64,75,76,77,78,79,80,81,82,83,84,87,88,89,90,91,92
0,16945,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,140,1,1,1,1,1,2,0,0,0,0,1380,1380,319,140,319,140,6900,61440,512,8192,0,0
1,16913,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,141,1,1,1,1,1,2,0,0,0,0,1380,1380,319,141,319,141,6900,61440,512,8192,0,0
2,16917,60,376,40,362,14,20,5,5,4,5,1,0,0,1,1,322,141,1,1,1,1,1,2,0,0,0,0,1380,1380,322,141,322,141,6900,61440,512,8192,0,0
3,17018,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,142,1,1,1,1,1,2,0,0,0,0,1380,1380,319,142,319,142,6900,61440,512,8192,0,0
4,16926,60,379,40,365,14,20,5,5,4,5,1,0,0,1,1,325,141,1,1,1,1,1,2,0,0,0,0,1380,1380,325,141,325,141,6900,61440,512,8192,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24858,3707,60,139,40,125,14,20,9,10,8,10,4,0,0,3,4,83,163,3,4,1,1,1,1,0,0,0,0,1460,1460,35,85,20,8,64240,49640,64077,49640,0,0
24859,58489,66,1061,52,1047,14,14,11,12,10,12,3,0,0,7,7,1197,318,7,7,1,1,1,1,0,0,0,0,1460,1460,995,73,6,21,5840,57920,5840,57920,0,0
24860,4582,66,163,52,149,14,14,10,13,9,13,4,0,0,5,7,97,258,5,7,1,1,1,1,0,0,0,0,1460,1460,35,97,6,8,57920,49232,57911,49232,0,0
24861,56031,60,590,40,576,14,20,28,52,27,52,6,0,0,1,45,119,23914,1,1,1,1,1,1,0,0,0,0,1460,1380,119,536,119,330,16080,61440,5840,8192,0,0


In [ ]:
classes = ['WWW','MAIL','FTP-CONTROL','FTP-PASV','ATTACK','P2P','DATABASE','FTP-DATA','MULTIMEDIA','SERVICES','INTERACTIVE','GAMES']
#classes = list(target.unique())
#classes

In [ ]:
#Converts the above list of classes to an array of index
target = target.apply(lambda x: classes.index(x))
target.unique()
target_test = target_test.apply(lambda x: classes.index(x))

In [ ]:
numeric_feature_names = ['ClientPort','10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']
numeric_features = df[numeric_feature_names] # Data for training
numeric_features_test = df_test[numeric_feature_names] # Data for testing
numeric_features.head()

,ClientPort,10,15,17,22,24,29,31,32,33,34,36,37,38,45,46,47,48,59,60,61,62,63,64,75,76,77,78,79,80,81,82,83,84,87,88,89,90,91,92
0,16945,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,140,1,1,1,1,1,2,0,0,0,0,1380,1380,319,140,319,140,6900,61440,512,8192,0,0
1,16913,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,141,1,1,1,1,1,2,0,0,0,0,1380,1380,319,141,319,141,6900,61440,512,8192,0,0
2,16917,60,376,40,362,14,20,5,5,4,5,1,0,0,1,1,322,141,1,1,1,1,1,2,0,0,0,0,1380,1380,322,141,322,141,6900,61440,512,8192,0,0
3,17018,60,373,40,359,14,20,5,5,4,5,1,0,0,1,1,319,142,1,1,1,1,1,2,0,0,0,0,1380,1380,319,142,319,142,6900,61440,512,8192,0,0
4,16926,60,379,40,365,14,20,5,5,4,5,1,0,0,1,1,325,141,1,1,1,1,1,2,0,0,0,0,1380,1380,325,141,325,141,6900,61440,512,8192,0,0


In [ ]:
#converting pandas dataframe to tensorflow object
numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features_test = tf.convert_to_tensor(numeric_features_test)
numeric_features

<tf.Tensor: shape=(24863, 40), dtype=int64, numpy=
array([[16945,    60,   373, ...,  8192,     0,     0],
       [16913,    60,   373, ...,  8192,     0,     0],
       [16917,    60,   376, ...,  8192,     0,     0],
       ...,
       [ 4582,    66,   163, ..., 49232,     0,     0],
       [56031,    60,   590, ...,  8192,     0,     0],
       [18275,    60,   130, ..., 49640,     0,     0]])>

In [ ]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)
#normalizer.adapt(numeric_features_test)

Ideia: Fazer a normalização entre 0 e 1 conforme o Andrews, aqui atualmente ele está centrado em 0 com desvio padrão em 1

In [ ]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='tanh'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

Requisição Weverton 
Como recuperar os pesos após os treinos? Descobrir o método

In [ ]:
model = get_basic_model()
model.fit(numeric_features, target, epochs=10, batch_size=BATCH_SIZE)

Epoch 1/10
2487/2487 [==============================] - 10s 4ms/step - loss: 0.4781 - accuracy: 0.8931
Epoch 2/10
2487/2487 [==============================] - 4s 2ms/step - loss: 0.1666 - accuracy: 0.9597
Epoch 3/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1273 - accuracy: 0.9660
Epoch 4/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1104 - accuracy: 0.9704
Epoch 5/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.1001 - accuracy: 0.9740
Epoch 6/10
2487/2487 [==============================] - 4s 1ms/step - loss: 0.0931 - accuracy: 0.9766
Epoch 7/10
2487/2487 [==============================] - 4s 1ms/step - loss: 0.0869 - accuracy: 0.9784
Epoch 8/10
2487/2487 [==============================] - 4s 1ms/step - loss: 0.0823 - accuracy: 0.9798
Epoch 9/10
2487/2487 [==============================] - 4s 1ms/step - loss: 0.0785 - accuracy: 0.9812
Epoch 10/10
2487/2487 [==============================] - 3s 1ms/step - loss: 0.07

In [ ]:
test_loss, test_acc = model.evaluate(numeric_features_test,  target_test, verbose=2, batch_size=BATCH_SIZE) 

print('\nTest accuracy:', test_acc)


2381/2381 - 2s - loss: 0.1574 - accuracy: 0.9590 - 2s/epoch - 981us/step

Test accuracy: 0.959035336971283


In [ ]:
print('\nPesos da primeira camada com inputs normalizados')
print(model.layers[0].weights)
print('\nPesos da segunda camada: intermediária')
print(model.layers[1].weights)
print('\nPesos da terceira camada: final')
print(model.layers[2].weights)


Pesos da primeira camada com inputs normalizados
[<tf.Variable 'mean:0' shape=(40,) dtype=float32, numpy=
array([1.8813523e+04, 6.2024391e+01, 7.0617371e+02, 4.4044239e+01,
       6.9216669e+02, 1.4007880e+01, 1.7980135e+01, 8.6330917e+01,
       5.7620537e+01, 8.5408112e+01, 5.7695206e+01, 3.5995174e+01,
       9.0656772e-02, 7.8832008e-02, 7.2039207e+01, 1.9789360e+01,
       3.9822301e+04, 1.3931234e+04, 5.3817296e+00, 5.9322238e+00,
       1.0045848e+00, 1.0461740e+00, 1.0000815e+00, 1.0902560e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.3084934e+03, 1.3489930e+03, 4.0109970e+02, 4.8949991e+02,
       2.2151576e+02, 2.7970181e+02, 2.1905771e+04, 3.8048879e+04,
       1.8475771e+04, 2.9452746e+04, 2.1558138e-02, 9.8942211e-03],
      dtype=float32)>, <tf.Variable 'variance:0' shape=(40,) dtype=float32, numpy=
array([4.7282010e+08, 8.2410641e+00, 1.8112305e+05, 3.2403339e+01,
       1.8113314e+05, 4.4662938e-02, 8.0480347e+00, 3.7168160e+06,
     